In [1]:
import brightway2 as bw
import bentso
from bentso.iterators import COUNTRIES
import wurst
from wurst.ecoinvent.electricity_markets import (
    move_all_generation_to_high_voltage, 
    remove_electricity_trade,
    all_providers,
)

/Users/cmutel/miniconda/envs/setac2019/lib/python3.7/site-packages/constructive_geometries/cg.py:77: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [2]:
bw.projects.set_current("SETAC 2019")

# How to handle ENTSO generation from `Other` and `Other renewable`?

Check to see how important they are in the ENTSO data.

You can set the key to a different value if you don't have the data cached already.

**Note**: To use `bentso`, you need to [register for a ENTSO-E API key](https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html) (see section on authentication).

In [3]:
c = bentso.CachingDataClient()

Using data directory /Users/cmutel/Library/Application Support/bentso


European countries where ENTSO doesn't have data

In [4]:
ENTSO_MISSING = {"HR", "CY", "IS", "LU", "ME", "NI"}

In [5]:
missing = ['Other renewable', 'Other']

In [6]:
for country in COUNTRIES:
    df = c.get_generation(country, 2017)
    if df is None:
        continue
    for label in missing:
        if label in df.columns:
            total = df[label].sum()
            if total > 0:
                print(country, label, total, total / df.sum().sum())

AT Other 770880.0 0.0032773083017291335
BE Other 5080873.0 0.06245732457602635
Value not in cache; returning nothing.
Value not in cache; returning nothing.
CZ Other renewable 2482996.0 0.030779251324929282
CZ Other 975209.0 0.012088703689145275
EE Other renewable 54694.0 0.004970315238886197
FI Other renewable 354906.0 0.005735598610298779
FI Other 1132735.0 0.018306011427918344
DE Other renewable 6365306.0 0.0029486705610072377
DE Other 165030109.0 0.0764487086855079
HU Other renewable 401251.0 0.0033896873540959706
HU Other 2970384.0 0.025093203709421297
IE Other 26671.0 0.0007093943672262622
Value not in cache; returning nothing.
IT Other 81423613.0 0.32047333882073326
LV Other 784921.0 0.10756488871189525
Value not in cache; returning nothing.
Value not in cache; returning nothing.
NL Other 6864.0 3.3435133670610246e-05
Value not in cache; returning nothing.
NO Other 380779.0 0.0025892372053025635
PT Other 318916.0 0.005834667717159087
RS Other 305419.0 0.008984141270862745
SK Oth

Traditionally, `Other renewable` is wind, solar, hydro, biofuels, tidal, and geothermal - but we have all of these covered. Therefore, we will set `Other renewable` to the country-specific mix of these renewables, as we don't have any better data.

`Other` is a tougher problem. Italy has an astonishingly high fraction, 0.32. We can compare the ENTSO numbers with more [details accounts from Terna, the Italian ISO](http://download.terna.it/terna/0000/1112/76.PDF) for 2016:

| ENTSO Label | ENTSO value (GWh) | Terna value (GWh) | Note |
| --- | --- | --- | --- |
| Biomass                         |  2.4 | 4.0 |
| Fossil Coal-derived gas         |  1.9 | 2.7 |
| Fossil Gas                      | 61.0 | 123 |
| Fossil Hard coal                | 10.0 | 32.4 |
| Fossil Oil                      |  1.2 | 3.8 |
| Geothermal                      |  5.8 | 5.8 |
| Hydro Pumped Storage            |  3.1 | 42.4 | All hydro |
| Hydro Run-of-river and poundage | 31.0 |  |
| Hydro Water Reservoir           |  6.3 |  |
| Other                           | 92.2 |  |
| Solar                           | 18.0 | 22.1 |
| Waste                           |   .3 | 15.5 |
| Wind Onshore                    | 17.1 | 17.7 |

It looks like the best we can do (without doing very detailed analysis for each country with `other`) is to assign the average mix of specific technologies to `Other`.

In [7]:
RENEWABLES = {
    'Solar',
    'Wind Onshore',
    'Wind Offshore',
    'Hydro Water Reservoir',
    'Hydro Run-of-river and poundage',
    'Marine',
    'Geothermal',
    'Biomass',
}

In [8]:
def drop_zero_columns(df):
    """Drop columns with zero generation"""
    for col in df:
        if not df[col].sum():
            df = df.drop(col, axis=1)
    return df

In [9]:
def remove_other_renewables(df):
    """Remove `Other renewables` column and rescale"""
    if 'Other renewable' in df:
        renewable_total = sum([df[label].sum() 
                               for label in RENEWABLES
                               if label in df])
        oth_renew_total = df['Other renewable'].sum()
        if not RENEWABLES.intersection(set(df.columns)):
            raise ValueError("No substitutable renewable sources found")
        
        for label in RENEWABLES:
            if label not in df:
                continue
            this = df[label].sum()
            multiplier = ((this / renewable_total) * (oth_renew_total / this)) + 1
            df[label] *= multiplier
        return df.drop('Other renewable', axis=1)
    else:
        return df

In [10]:
def remove_others(df):
    """Remove `Other` column and rescale"""
    if 'Other' in df:
        total = df.sum().sum()
        scale = total / (total - df['Other'].sum())
        return df.drop('Other', axis=1) * scale
    else:
        return df

Simple test to make sure we aren't losing generation

In [11]:
df = c.get_generation("DE", 2017)

In [12]:
df.sum(), df.sum().sum()

(Biomass                            160838272.0
 Fossil Brown coal/Lignite          517801091.0
 Fossil Coal-derived gas             15071396.0
 Fossil Gas                          67615776.0
 Fossil Hard coal                   262629839.0
 Fossil Oil shale                           0.0
 Fossil Peat                                0.0
 Geothermal                            139192.0
 Hydro Pumped Storage                36986326.0
 Hydro Run-of-river and poundage     59930085.0
 Hydro Water Reservoir                2426830.0
 Marine                                     0.0
 Nuclear                            288840518.0
 Other                              165030109.0
 Other renewable                      6365306.0
 Solar                              143529219.0
 Waste                               20879488.0
 Wind Offshore                       69675245.0
 Wind Onshore                       340944991.0
 dtype: float64, 2158703683.0)

In [13]:
df = remove_other_renewables(remove_others(drop_zero_columns(df)))

In [14]:
df.sum(), df.sum().sum()

(Biomass                            1.755778e+08
 Fossil Brown coal/Lignite          5.606631e+08
 Fossil Coal-derived gas            1.631896e+07
 Fossil Gas                         7.321280e+07
 Fossil Hard coal                   2.843695e+08
 Geothermal                         1.519478e+05
 Hydro Pumped Storage               4.004794e+07
 Hydro Run-of-river and poundage    6.542218e+07
 Hydro Water Reservoir              2.649229e+06
 Nuclear                            3.127498e+08
 Solar                              1.566825e+08
 Waste                              2.260783e+07
 Wind Offshore                      7.606040e+07
 Wind Onshore                       3.721898e+08
 dtype: float64, 2158703683.0000024)

The cleaning and missing country functionality was integrated into bentso version 0.2.

# Mapping ENTSO-E labels to ecoinvent

List all ecoinvent activities that have the unit `kilowatt hour`:

In [15]:
sorted({x['name'] for x in bw.Database("ecoinvent 3.5 cutoff") if x['unit'] == 'kilowatt hour'})

['biogas, burned in micro gas turbine 100kWe',
 'biogas, burned in polymer electrolyte membrane fuel cell 2kWe, future',
 'biogas, burned in solid oxide fuel cell 125kWe, future',
 'biogas, burned in solid oxide fuel cell, with micro gas turbine, 180kWe, future',
 'burnt shale production',
 'cane sugar production with ethanol by-product',
 'electricity production, coal, aluminium industry',
 'electricity production, deep geothermal',
 'electricity production, hard coal',
 'electricity production, hard coal, at coal mine power plant',
 'electricity production, hard coal, conventional',
 'electricity production, hard coal, supercritical',
 'electricity production, high voltage, for Swiss Federal Railways',
 'electricity production, hydro, aluminium industry',
 'electricity production, hydro, pumped storage',
 'electricity production, hydro, reservoir, alpine region',
 'electricity production, hydro, reservoir, alpine region, label-certified',
 'electricity production, hydro, reservoir, n

Manual match of ecoinvent technologies

In [16]:
generic_mapping = {
    'Solar': [
        'electricity production, photovoltaic, 3kWp facade installation, multi-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp facade installation, multi-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp facade installation, single-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp facade installation, single-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp flat-roof installation, multi-Si',
        'electricity production, photovoltaic, 3kWp flat-roof installation, single-Si',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, CIS, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, CdTe, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, a-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted, label-certified',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, ribbon-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, ribbon-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, laminated, integrated',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted',
        'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted, label-certified',
        'electricity production, photovoltaic, 570kWp open ground installation, multi-Si',
        'electricity production, solar thermal parabolic trough, 50 MW',
        'electricity production, solar tower power plant, 20 MW',        
    ],
    'Wind Onshore': [
        'electricity production, wind, 1-3MW turbine, onshore',
        'electricity production, wind, 1-3MW turbine, onshore, label-certified',
        'electricity production, wind, 2.3MW turbine, precast concrete tower, onshore',
        'electricity production, wind, <1MW turbine, onshore',
        'electricity production, wind, <1MW turbine, onshore, label-certified',
        'electricity production, wind, >3MW turbine, onshore',
     ],
    'Wind Offshore': [
        'electricity production, wind, 1-3MW turbine, offshore'
    ],
    'Hydro Water Reservoir': [
        'electricity production, hydro, reservoir, alpine region',
        'electricity production, hydro, reservoir, alpine region, label-certified',
        'electricity production, hydro, reservoir, non-alpine region',
        'electricity production, hydro, reservoir, tropical region',
        'electricity production, hydro, reservoir, alpine region, label-certified',
    ],
    'Hydro Run-of-river and poundage': [
        'electricity production, hydro, run-of-river',
        'electricity production, hydro, run-of-river, label-certified',
    ],
    'Waste': [
        'electricity, from municipal waste incineration to generic market for electricity, medium voltage'
    ],
    'Marine': [
        'Production of electricity by tide, wave, ocean'
    ],
    'Geothermal': [
        'electricity production, deep geothermal'
    ],
    'Nuclear': [
        'electricity production, nuclear, boiling water reactor',
        'electricity production, nuclear, pressure water reactor, heavy water moderated',
        'electricity production, nuclear, pressure water reactor',
    ],
    'Fossil Brown coal/Lignite': [
        'electricity production, lignite',
        'heat and power co-generation, lignite',
    ],
    'Fossil Hard coal': [
        'electricity production, hard coal',
        'electricity production, lignite',
        'electricity production, hard coal, conventional',
        'electricity production, hard coal, supercritical',
        'heat and power co-generation, hard coal',
    ],
    'Fossil Gas': [
        'electricity production, natural gas, 10MW',
        'electricity production, natural gas, combined cycle power plant',
        'electricity production, natural gas, conventional power plant',
        'natural gas, burned in micro gas turbine, 100kWe',
        'heat and power co-generation, natural gas, 160kW electrical, Jakobsberg',
        'heat and power co-generation, natural gas, 160kW electrical, lambda=1',
        'heat and power co-generation, natural gas, 1MW electrical, lean burn',
        'heat and power co-generation, natural gas, 200kW electrical, lean burn',
        'heat and power co-generation, natural gas, 500kW electrical, lean burn',
        'heat and power co-generation, natural gas, 50kW electrical, lean burn',
        'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical',
        'heat and power co-generation, natural gas, conventional power plant, 100MW electrical',
        'heat and power co-generation, natural gas, mini-plant 2KW electrical',
    ],
    'Hydro Pumped Storage': [
        'electricity production, hydro, pumped storage'
    ],
    'Fossil Oil': [
        'electricity production, oil',
        'heat and power co-generation, oil',
    ],
    'Fossil Peat': [
        'electricity production, peat'
    ],
    'Fossil Oil shale': [
        'electricity production, oil', # Ecoinvent assumption, no activity for burning shale
        # 'burnt shale production'     # This is only production, not combustion
    ],
    'Biomass': [
        'heat and power co-generation, biogas, gas engine',
        'heat and power co-generation, biogas, gas engine, label-certified',
        'heat and power co-generation, wood chips, 2000 kW',
        'heat and power co-generation, wood chips, 2000 kW, state-of-the-art 2014',
        'heat and power co-generation, wood chips, 6667 kW',
        'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
        'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014, label-certified',
        'wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future',
        'electricity production, wood, future',
        'biogas, burned in micro gas turbine 100kWe',
    ],
    'Fossil Coal-derived gas': [
        'treatment of coal gas, in power plant'
    ],
}

Use `wurst` to extract data into an easily usable form, and move all generation to high voltage mixes.

In [17]:
db = wurst.extract_brightway2_databases(["ecoinvent 3.5 cutoff"])

Getting activity data


  0%|          | 0/544735 [00:00<?, ?it/s]

Adding exchange data to activities


100%|██████████| 544735/544735 [00:28<00:00, 18910.54it/s]


Filling out exchange data


100%|██████████| 16022/16022 [00:02<00:00, 6590.43it/s]


In [18]:
db = move_all_generation_to_high_voltage(db)
db = remove_electricity_trade(db)

There are multiple possible ecoinvent technologies for most ENTSO-E labels. We will take disaggregation factors from the ecoinvent country-specific mixes, and use German generation as a back-up.

In [19]:
mix_dictionary = {ds['location']: ds for ds in db 
                  if ds['name'] == 'market for electricity, high voltage'}

In [20]:
def has_any(entso_label, country, db):
    """Check if ``country`` has any ecoinvent activities which meet 
    generation type ``entso_label`` in their electricity grid mix"""
    mix = mix_dictionary[country]
    possibles = generic_mapping[entso_label]
    return any(exc['name'] in possibles for exc in mix['exchanges'])

In [21]:
def generation_in_country_exists(possibles, country, db):
    """Check if any of the ``possibles`` generaters exist in the ``country``"""
    return any(ds['name'] in possibles
               for ds in db
               if ds['location'] == country)

In [22]:
for country in set(COUNTRIES).difference(ENTSO_MISSING):
    generation = c.get_generation(country, 2017)
    for label in generation.columns:
        if label in ("Other", "Other renewable"):
            continue
        if not generation[label].sum():
            continue
        if not has_any(label, country, db):
            print(label, 
                  country, 
                  generation_in_country_exists(generic_mapping[label], country, db), 
                  generation_in_country_exists(generic_mapping[label], 'DE', db), 
                  generation[label].sum() / generation.sum().sum())

Hydro Water Reservoir GR False True 0.06809651584952488
Hydro Water Reservoir PL False True 0.0009553833617670647
Hydro Run-of-river and poundage NO False True 0.07910015318617056
Hydro Water Reservoir RO False True 0.0716532753014878
Biomass RS True True 0.0009410699646631053
Hydro Water Reservoir BG False True 0.05062257710151984
Waste BG True True 0.000245621689421549
Solar EE False True 0.0007427210744764852
Hydro Water Reservoir HU False True 0.0034863301620412664
Hydro Pumped Storage CH True True 0.18196568948794153
Wind Onshore SI True True 0.00029687910074623796


# Create correspondence tables

Our correspondence tables will follow the simple schema from [correspondentia](https://github.com/BONSAMURAIS/correspondentia):

```python

(from_label, to_label, weight)

```

In [23]:
backup = mix_dictionary['DE']

In [24]:
def get_ct_for_country(country):
    table = []

    df = c.get_generation(country, 2017)
    df = remove_other_renewables(remove_others(drop_zero_columns(df)))
    mix = mix_dictionary[country]
    
    for col in df:
        exchanges = [exc for exc in mix['exchanges'] if exc['name'] in generic_mapping[col]]
        if not exchanges:
            exchanges = [exc for exc in backup['exchanges'] if exc['name'] in generic_mapping[col]]
            
        subtotal = sum([exc['amount'] for exc in exchanges])
        for exc in exchanges:
            table.append((col, exc['name'], exc['amount'] / subtotal))
    
    return table

## Write correspondence table as a [tabular data package](https://frictionlessdata.io/specs/tabular-data-package/).

In [25]:
import csv
HEADERS = ["from_label", "to_label", "weight"]


def write_csv(filepath, data):
    with open(filepath, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(HEADERS)
        for row in data:
            writer.writerow(row)    

In [26]:
import json


def write_metadata(filepath, name):
    metadata = {
        "profile": "tabular-data-package",
        "name": name,
        "resources": [{
            "path": filepath.name,
            "encoding": "utf-8",
            "format": "csv",
            "mediatype": "text/csv",
            "profile": "tabular-data-resource",
            "schema": {
                "fields": [
                    {
                        "name": "from_label",
                        "type": "string"
                    },
                    {
                        "name": "to_label",
                        "type": "string"
                    },
                    {
                        "name": "weight",
                        "type": "number"
                    }
                ]
            }
        }]
    }
    with open(filepath.parent / "datapackage.json", "w") as f:
        json.dump(metadata, f)

In [27]:
from pathlib import Path
import os

def write_datapackage(data, dirname, name, base_dir=None):
    if base_dir is None:
        base_dir = os.getcwd()
    dirpath = (Path(base_dir) / dirname).resolve()
    if dirpath.is_dir():
        raise ValueError("Directory already exists")
    
    dirpath.mkdir()
    filepath = dirpath / "corr_table.csv"
    write_csv(filepath, data)
    write_metadata(filepath, name)

In [29]:
for country in set(COUNTRIES).difference(ENTSO_MISSING):
    write_datapackage(
        get_ct_for_country(country),
        country,
        "{}-ct-entso-ecoinvent35".format(country),
        base_dir="corr_tables"
    )